# Pickett Plot

This notebook demonstrates how to create Pickett plots using GeoSuite's `pickett_plot()` function.

## Overview

Pickett plots are log-log crossplots of porosity vs resistivity used to:
- Determine water saturation (Sw)
- Identify reservoir quality zones
- Distinguish pay from non-pay intervals

GeoSuite uses **matplotlib** with **signalplot** styling for clean, professional plots.

This notebook will show you how to:

1. Load well log data
2. Create Pickett plots with `pickett_plot()`
3. Interpret saturation isolines
4. Customize plot parameters

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs

print("GeoSuite imported successfully!")

## 1. Load Well Log Data

Load demo well log data that includes porosity and resistivity measurements.

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")

# Check for porosity and resistivity columns
porosity_cols = [col for col in df.columns if 'PHI' in col.upper() or 'NPHI' in col.upper() or 'POR' in col.upper()]
resistivity_cols = [col for col in df.columns if 'RT' in col.upper() or 'RES' in col.upper() or 'ILD' in col.upper()]

print(f"\nPorosity columns found: {porosity_cols}")
print(f"Resistivity columns found: {resistivity_cols}")

df.head()

## 2. Create Pickett Plot

Use GeoSuite's `pickett_plot()` function to create a professional Pickett plot with saturation isolines.

In [ ]:
# Determine which columns to use
porosity_col = porosity_cols[0] if porosity_cols else df.columns[1]
resistivity_col = resistivity_cols[0] if resistivity_cols else df.columns[2]

print(f"Using porosity column: {porosity_col}")
print(f"Using resistivity column: {resistivity_col}")

# Create Pickett plot with default parameters
# Default: m=2.0, n=2.0, a=1.0, rw=0.05
fig = pickett_plot(
    df=df,
    porosity_col=porosity_col,
    resistivity_col=resistivity_col,
    title='Pickett Plot - Demo Well',
    show_water_line=True,  # Show Sw = 100% line
    show_sw_lines=True     # Show Sw = 50% and 25% lines
)

plt.show()

## 3. Customize Archie Parameters

Adjust the Archie equation parameters (m, n, a, Rw) to match your formation characteristics.

In [ ]:
# Create Pickett plot with custom Archie parameters
# These are typical for sandstone formations
fig = pickett_plot(
    df=df,
    porosity_col=porosity_col,
    resistivity_col=resistivity_col,
    m=2.0,      # Cementation exponent (sandstone: 1.8-2.2)
    n=2.0,      # Saturation exponent (typically ~2.0)
    a=1.0,      # Tortuosity factor (sandstone: 0.6-1.4)
    rw=0.05,    # Formation water resistivity (ohm-m) - adjust for your area
    title='Pickett Plot - Custom Parameters',
    show_water_line=True,
    show_sw_lines=True
)

plt.show()

## 4. Calculate Water Saturation

Use GeoSuite's `calculate_water_saturation()` to compute Sw values for each depth point.

In [ ]:
# Calculate water saturation using GeoSuite's calculate_water_saturation()
# This uses the same Archie equation as the Pickett plot
df['SW'] = calculate_water_saturation(
    phi=df[porosity_col].values,
    rt=df[resistivity_col].values,
    rw=0.05,  # Formation water resistivity
    m=2.0,    # Cementation exponent
    n=2.0,    # Saturation exponent
    a=1.0     # Tortuosity factor
)

print("Water Saturation Statistics:")
print(df['SW'].describe())

# Identify pay zones (typically Sw < 0.5 or 50%)
df['Pay_Zone'] = df['SW'] < 0.5
print(f"\nPay zone intervals: {df['Pay_Zone'].sum()} points ({df['Pay_Zone'].sum()/len(df)*100:.1f}%)")

## 5. Summary

This notebook demonstrated:

-  Creating Pickett plots with `pickett_plot()` (matplotlib with signalplot styling)
-  Customizing Archie parameters (m, n, a, Rw)
-  Calculating water saturation with `calculate_water_saturation()`
-  Identifying pay zones from saturation values

### Note on Plotting

GeoSuite's plotting functions use **matplotlib** with **signalplot** styling for clean, professional visualizations. The `signalplot` library provides minimalist, publication-ready styling automatically applied to all plots.

### Interpreting Pickett Plots

- **Points below Sw=100% line**: Hydrocarbon-bearing zones
- **Points near Sw=50% line**: Transition zones
- **Points near Sw=100% line**: Water-bearing zones
- **Points far right (high porosity)**: Good reservoir quality
- **Points far left (low porosity)**: Tight/non-reservoir

### Next Steps

- Use `geosuite.petro.buckles_plot()` for Bulk Volume Water analysis
- Use `geosuite.petro.calculate_porosity_from_density()` for density porosity
- Use `geosuite.plotting.create_strip_chart()` to visualize logs with saturation